In [2]:
#Imports
import pandas as pd
import numpy as np
from helpers.data_analysis import TweetsAnalyzer, BTCAnalyzer

In [3]:
start_time = pd.Timestamp('2017-08-17 04:00:00+00:00')
end_time = pd.Timestamp('2019-11-23 14:00:20+00:00')
print(start_time- end_time)

-829 days +13:59:40


## BTC

In [8]:
# BTC data loading
btc_clean = BTCAnalyzer(file_path='clean_data/btc.parquet')

########## Basic information ##########
- Shape of dataset: (46028, 2)
- Average time granularity: 0 days 01:00:00
- Data starting from 2017-08-17 04:00:00+00:00 to 2022-11-16 23:00:00+00:00
- Data Types of each column:
close      float32
returns    float32
dtype: object


In [9]:
df_btc=btc_clean.data
df_btc = df_btc.loc[start_time:end_time]
df_btc

,close,returns
open_time,,
2017-08-17 04:00:00+00:00,4308.830078,NaN
2017-08-17 05:00:00+00:00,4315.319824,0.001506
2017-08-17 06:00:00+00:00,4324.350098,0.002093
2017-08-17 07:00:00+00:00,4349.990234,0.005929
2017-08-17 08:00:00+00:00,4360.689941,0.002460
...,...,...
2019-11-23 10:00:00+00:00,7165.169922,0.001634
2019-11-23 11:00:00+00:00,7171.919922,0.000942
2019-11-23 12:00:00+00:00,7150.140137,-0.003037


In [7]:
#check the date
# Ensure the 'open_time' column is in datetime format
# Ensure the index is in datetime format
df_btc.index = pd.to_datetime(df_btc.index)

# Create a complete range of hourly timestamps
full_range = pd.date_range(start=df_btc.index.min(), end=df_btc.index.max(), freq='h')

# Find missing timestamps
missing_hours = full_range.difference(df_btc.index)

# Print the missing hours
print(f"Missing hours: {missing_hours}")

Missing hours: DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='h')


In [5]:
# Checking the number of returns that are less than 0.001 because I need to determine the threshold for the neutral label
df_btc[np.abs(df_btc['returns'])<=0.001].shape

(3819, 8)

In [6]:
# Function to apply a label using the returns
def label_btc(returns, threshold):
    if returns > threshold:
        return 'BULLISH'
    elif returns < -threshold:
        return 'BEARISH'
    else:
        return 'NEUTRAL'

In [7]:
# Apply the label te each hour
df_btc['label'] = df_btc['returns'].apply(lambda x: label_btc(x, 0.001))
mapping = {'BEARISH':0,'NEUTRAL': 1,'BULLISH': 2}
df_btc['final_label'] = df_btc['label'].map(mapping)
df_btc

C:\Users\ratas\AppData\Local\Temp\ipykernel_4816\1285183844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_btc['label'] = df_btc['returns'].apply(lambda x: label_btc(x, 0.001))
C:\Users\ratas\AppData\Local\Temp\ipykernel_4816\1285183844.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_btc['final_label'] = df_btc['label'].map(mapping)


,open,high,low,close,volume,quote_asset_volume,number_of_trades,returns,label,final_label
open_time,,,,,,,,,,
2017-08-17 04:00:00+00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,2.023661e+05,171,NaN,NEUTRAL,1
2017-08-17 05:00:00+00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1.003048e+05,102,0.001506,BULLISH,2
2017-08-17 06:00:00+00:00,4315.319824,4345.450195,4309.370117,4324.350098,7.229691,3.128231e+04,36,0.002093,BULLISH,2
2017-08-17 07:00:00+00:00,4324.350098,4349.990234,4287.410156,4349.990234,4.443249,1.924106e+04,25,0.005929,BULLISH,2
2017-08-17 08:00:00+00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,4.239503e+03,28,0.002460,BULLISH,2
...,...,...,...,...,...,...,...,...,...,...
2019-11-23 10:00:00+00:00,7151.560059,7199.990234,7114.279785,7165.169922,2282.374512,1.634108e+07,19421,0.001634,BULLISH,2
2019-11-23 11:00:00+00:00,7166.379883,7215.000000,7149.220215,7171.919922,2215.600586,1.591635e+07,18431,0.000942,NEUTRAL,1
2019-11-23 12:00:00+00:00,7171.919922,7197.229980,7139.750000,7150.140137,2049.657715,1.469170e+07,19657,-0.003037,BEARISH,0


In [8]:
# Checking the distribution of the labels
print(df_btc.shape)
df_btc['label'].value_counts()

(19788, 10)


label
BULLISH    8187
BEARISH    7781
NEUTRAL    3820
Name: count, dtype: int64

## Twitter

In [10]:
# Loading the twitter data
twitter_clean = TweetsAnalyzer(file_path='clean_data/twitter.parquet')
df_twitter=twitter_clean.data
df_twitter.set_index('timestamp', inplace = True)
df_twitter = df_twitter.loc[start_time:end_time]

########## Basic information ##########
- Shape of dataset: (59326, 14)
- Data starting from 2009-01-11 03:33:52+00:00 to 2019-11-23 14:00:20+00:00
- Data Types of each column:
index                            int64
id                             float64
user                            object
fullname                        object
url                             object
timestamp          datetime64[ns, UTC]
replies                          int64
likes                            int64
retweets                         int64
text                            object
cleaned_text                    object
lang                            object
sentiment                       object
sentiment_score                float64
dtype: object


In [6]:
df_twitter

,index,id,user,fullname,url,replies,likes,retweets,text,cleaned_text,lang,sentiment,sentiment_score
timestamp,,,,,,,,,,,,,
2017-08-17 05:03:00+00:00,2843339,8.980777e+17,VivoBitcoin,Vivo Bitcoin,/VivoBitcoin/status/898077687771803648,5,44,203,#BTC 24hr Summary:\nLast: $4310.00\nHigh: $439...,btc 24hr summary: last: $4310.00 high: $4397.0...,en,NEUTRAL,0.659304
2017-08-17 14:01:19+00:00,8190436,8.981830e+17,BitPay,BitPay,None,140,97,51,What Bitcore Users Need to Know To Be Ready fo...,what bitcore users need to know to be ready fo...,en,NEUTRAL,0.924089
2017-08-17 16:07:47+00:00,11453726,8.982148e+17,blockchain,Blockchain,None,54,480,243,"Ether has arrived. Seamlessly store, send, rec...","ether has arrived. seamlessly store, send, rec...",en,BULLISH,0.838789
2017-08-18 13:15:08+00:00,13966462,8.985337e+17,AngeloBTC,Angelo฿TC,None,2,134,15,"I currently have no bias on $BTC. However, I a...","i currently have no bias on $btc. however, i a...",en,BULLISH,0.480497
2017-08-18 18:55:49+00:00,450744,8.986497e+17,BrianRoemmele,Brian Roemmele,None,1,366,167,BOOM! \n\nBitcoin Cash BCH $645.15🚀\n\nMiners ...,boom! bitcoin cash bch $645.15🚀 miners are mak...,en,BULLISH,0.993328
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-23 11:55:10+00:00,16879539,1.198208e+18,qhfofficial,Quantum Hedge Fund Official,None,0,101,14,📣Remind you📣\n\nThere are two2️⃣ days left unt...,📣remind you📣 there are two2️⃣ days left until ...,en,BULLISH,0.984112
2019-11-23 12:38:31+00:00,16881465,1.198219e+18,matt_odell,Matt Odell,None,8,119,27,The world is ready for sats to be the standard...,the world is ready for sats to be the standard...,en,BULLISH,0.861456
2019-11-23 12:41:58+00:00,16881483,1.198220e+18,CryptoMichNL,Crypto Michaël,None,9,149,45,"$BTC #BITCOIN\n\nAll in all, the long-term tre...","$btc bitcoin all in all, the long-term trend i...",en,BULLISH,0.984184


In [7]:
# Checking the distribution of the sentiment
print(df_twitter['sentiment'].value_counts())
print(df_twitter.groupby('sentiment')['sentiment_score'].mean())
print(df_twitter.shape)

sentiment
BULLISH    26143
BEARISH    16299
NEUTRAL    16166
Name: count, dtype: int64
sentiment
BEARISH    0.834565
BULLISH    0.803431
NEUTRAL    0.770533
Name: sentiment_score, dtype: float64
(58608, 13)


In [12]:
# Number of unique hours in the dataset
h = df_twitter
h['time'] = pd.to_datetime(df_twitter.index).floor('h')
print(h['time'].nunique())

10579


C:\Users\ratas\AppData\Local\Temp\ipykernel_4816\1508761858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h['time'] = pd.to_datetime(df_twitter.index).floor('h')


In [13]:
def label(rate, threshold):
    if rate > threshold:
        return 'BULLISH'
    elif rate < -threshold:
        return 'BEARISH'
    else:
        return 'NEUTRAL'

def weighted_score(tweets, granularity='h'):
    mapped_sentiment = {'BEARISH': -1, 'NEUTRAL': 0, 'BULLISH': 1}
    tweets.loc[:, 'mapped_sentiment'] = tweets['sentiment'].map(mapped_sentiment)
    tweets.loc[:, 'weighted_sentiment'] = tweets['mapped_sentiment'] * tweets['sentiment_score']
    tweets.loc[:, 'counts'] = 1
    tweets = tweets.resample(granularity).agg({
        'weighted_sentiment': 'mean',
        'counts': 'sum'
    }).dropna()
    tweets['final_sentiment'] = tweets['weighted_sentiment'].apply(lambda x: label(x, 0.33))
    return tweets
"""
def apply_label(tweet):
    if tweet['counts'] < 5:
        return label(tweet['weighted_sentiment'], 0.16)
    else:
        return label(tweet['weighted_sentiment'], 0.33)

def label_sentiment(tweets):
    tweets.loc[:,'final_sentiment'] = tweets.apply(apply_label, axis=1)
"""
tweets_ts = weighted_score(df_twitter)
#label_sentiment(Tweets_TS)
tweets_ts

C:\Users\ratas\AppData\Local\Temp\ipykernel_4816\383515743.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets.loc[:, 'mapped_sentiment'] = tweets['sentiment'].map(mapped_sentiment)
C:\Users\ratas\AppData\Local\Temp\ipykernel_4816\383515743.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets.loc[:, 'weighted_sentiment'] = tweets['mapped_sentiment'] * tweets['sentiment_score']
C:\Users\ratas\AppData\Local\Temp\ipykernel_4816\383515743.py:13: SettingWithCopyWarning: 
A value is trying to be se

,weighted_sentiment,counts,final_sentiment
timestamp,,,
2017-08-17 05:00:00+00:00,0.000000,1,NEUTRAL
2017-08-17 14:00:00+00:00,0.000000,1,NEUTRAL
2017-08-17 16:00:00+00:00,0.838789,1,BULLISH
2017-08-18 13:00:00+00:00,0.480497,1,BULLISH
2017-08-18 18:00:00+00:00,0.993328,1,BULLISH
...,...,...,...
2019-11-23 10:00:00+00:00,0.510216,3,BULLISH
2019-11-23 11:00:00+00:00,0.454179,3,BULLISH
2019-11-23 12:00:00+00:00,0.922820,2,BULLISH


In [14]:
# Checking the distribution of the final sentiment
print(tweets_ts['final_sentiment'].value_counts())
print(tweets_ts['final_sentiment'].value_counts().sum())

final_sentiment
NEUTRAL    5927
BULLISH    3302
BEARISH    1350
Name: count, dtype: int64
10579


In [15]:
# Mapping 0 -> Bearish, 1 -> Neutral, 2 -> Bullish
mapping = {'BEARISH': 0,'NEUTRAL': 1,'BULLISH': 2}
tweets_ts['final_sentiment'] = tweets_ts['final_sentiment'].map(mapping)
tweets_ts

,weighted_sentiment,counts,final_sentiment
timestamp,,,
2017-08-17 05:00:00+00:00,0.000000,1,1
2017-08-17 14:00:00+00:00,0.000000,1,1
2017-08-17 16:00:00+00:00,0.838789,1,2
2017-08-18 13:00:00+00:00,0.480497,1,2
2017-08-18 18:00:00+00:00,0.993328,1,2
...,...,...,...
2019-11-23 10:00:00+00:00,0.510216,3,2
2019-11-23 11:00:00+00:00,0.454179,3,2
2019-11-23 12:00:00+00:00,0.922820,2,2


In [16]:
(tweets_ts['weighted_sentiment']==0).sum()

np.int64(1454)

In [17]:
# Merging the two datasets
merged_df = df_btc.merge(tweets_ts, left_index=True, right_index=True, how='left')
merged_df

,open,high,low,close,volume,quote_asset_volume,number_of_trades,returns,label,final_label,weighted_sentiment,counts,final_sentiment
open_time,,,,,,,,,,,,,
2017-08-17 04:00:00+00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,2.023661e+05,171,NaN,NEUTRAL,1,NaN,NaN,NaN
2017-08-17 05:00:00+00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1.003048e+05,102,0.001506,BULLISH,2,0.000000,1.0,1.0
2017-08-17 06:00:00+00:00,4315.319824,4345.450195,4309.370117,4324.350098,7.229691,3.128231e+04,36,0.002093,BULLISH,2,NaN,NaN,NaN
2017-08-17 07:00:00+00:00,4324.350098,4349.990234,4287.410156,4349.990234,4.443249,1.924106e+04,25,0.005929,BULLISH,2,NaN,NaN,NaN
2017-08-17 08:00:00+00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,4.239503e+03,28,0.002460,BULLISH,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-23 10:00:00+00:00,7151.560059,7199.990234,7114.279785,7165.169922,2282.374512,1.634108e+07,19421,0.001634,BULLISH,2,0.510216,3.0,2.0
2019-11-23 11:00:00+00:00,7166.379883,7215.000000,7149.220215,7171.919922,2215.600586,1.591635e+07,18431,0.000942,NEUTRAL,1,0.454179,3.0,2.0
2019-11-23 12:00:00+00:00,7171.919922,7197.229980,7139.750000,7150.140137,2049.657715,1.469170e+07,19657,-0.003037,BEARISH,0,0.922820,2.0,2.0


In [18]:
# filling the missing values with neutral sentiment (or 1)
merged_df['final_sentiment'] = merged_df['final_sentiment'].fillna(1)
merged_df

,open,high,low,close,volume,quote_asset_volume,number_of_trades,returns,label,final_label,weighted_sentiment,counts,final_sentiment
open_time,,,,,,,,,,,,,
2017-08-17 04:00:00+00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,2.023661e+05,171,NaN,NEUTRAL,1,NaN,NaN,1.0
2017-08-17 05:00:00+00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1.003048e+05,102,0.001506,BULLISH,2,0.000000,1.0,1.0
2017-08-17 06:00:00+00:00,4315.319824,4345.450195,4309.370117,4324.350098,7.229691,3.128231e+04,36,0.002093,BULLISH,2,NaN,NaN,1.0
2017-08-17 07:00:00+00:00,4324.350098,4349.990234,4287.410156,4349.990234,4.443249,1.924106e+04,25,0.005929,BULLISH,2,NaN,NaN,1.0
2017-08-17 08:00:00+00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,4.239503e+03,28,0.002460,BULLISH,2,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-23 10:00:00+00:00,7151.560059,7199.990234,7114.279785,7165.169922,2282.374512,1.634108e+07,19421,0.001634,BULLISH,2,0.510216,3.0,2.0
2019-11-23 11:00:00+00:00,7166.379883,7215.000000,7149.220215,7171.919922,2215.600586,1.591635e+07,18431,0.000942,NEUTRAL,1,0.454179,3.0,2.0
2019-11-23 12:00:00+00:00,7171.919922,7197.229980,7139.750000,7150.140137,2049.657715,1.469170e+07,19657,-0.003037,BEARISH,0,0.922820,2.0,2.0


In [19]:
# Checking for missing values
merged_df.isnull().sum()

open                     0
high                     0
low                      0
close                    0
volume                   0
quote_asset_volume       0
number_of_trades         0
returns                  1
label                    0
final_label              0
weighted_sentiment    9244
counts                9244
final_sentiment          0
dtype: int64

In [20]:
# Tweets time series array
tweets_time_series = merged_df['final_sentiment'].astype(int).to_numpy()
print(tweets_time_series.shape)

(19788,)


In [21]:
# BTC time series array
btc_time_series = merged_df['final_label'].astype(int).to_numpy()
print(btc_time_series.shape)

(19788,)


In [23]:
btc_time_series

array([1, 2, 2, ..., 0, 2, 2])

In [24]:
tweets_time_series

array([1, 1, 1, ..., 2, 2, 2])

In [25]:
# Saving the data
np.save('data/btc_time_series.npy', btc_time_series)
np.save('data/tweets_time_series.npy', tweets_time_series)